# Obteniendo el número de muertes confirmadas por COVID-19 por **semana epidemiológica** de cada uno de los **24 departamentos** del Perú

## 0. Cargar librerías

In [126]:
import pandas as pd
import numpy as np

import functions as fn

## 1. Cargamos direcciones de RawData

In [127]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

## 2. Procesamos el dataset de fallecidos

### 2.1. Añadimos el año, semana epidemiológica y departamento de cada fallecido

In [128]:
def just_cities(fal_url):
    """
    Función para filtrar a cada fallecido reportado en su año
    y semana epidemiológica por depatamento (Fallecidos = fal)
    """         
    fal_col = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO']
    df_fal = fn.read_largeCSV_file(fal_url, ';', fal_col)
    fn.variable_fecha(df_fal, 'FECHA_FALLECIMIENTO')
    fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO')
    df_fal['fallecido'] = 1     # To count cases                                   
    df_fal['fallecido'].apply(np.int8) 
    
    return df_fal

In [129]:
falxdep = just_cities(fal_url)

In [69]:
print(falxdep.head())

  DEPARTAMENTO  epi_year  epi_week  fallecido
0   LAMBAYEQUE      2021        23          1
1        PIURA      2021        11          1
2          ICA      2021        22          1
3     AREQUIPA      2021        26          1
4         LIMA      2021        18          1


**IMPORTANTE:** No tenemos el total de fallecidos por departamento y semana epidemiológica, sino el departamento y semana epidemiológica de cada caso.

### 2.2. Realizamos una crosstab de fallecidos por COVID-19 por departamento de Perú, para cada semana epidemiológica y año

Esto para obtener el TOTAL de fallecidos por cada departamento y semana epidemiológica

In [70]:
ct_falxdep = pd.crosstab(index=[falxdep['epi_year'], falxdep['epi_week']],
                       columns=[falxdep['fallecido'], falxdep['DEPARTAMENTO']],
                       margins = False)

In [71]:
print(ct_falxdep.head())

fallecido                1                  ...                     
DEPARTAMENTO      AMAZONAS ANCASH APURIMAC  ... TACNA TUMBES UCAYALI
epi_year epi_week                           ...                     
2020     10              0      0        0  ...     0      0       0
         11              0      0        0  ...     0      0       0
         12              0      0        0  ...     0      0       2
         13              0      4        0  ...     0      0       3
         14              0      0        0  ...     1      3      11

[5 rows x 25 columns]


Ahora tenemos el TOTAL de fallecidos por cada departamento y semana epidemiológica

In [72]:
ct_falxdep.to_csv('Data/fallecidos_x_departamentos_x_semanasEpi.csv')

### 2.3. Arreglamos los índices
(Buscar una manera más eficiente de hacerlo, en lugar de guardar y leer el dataset nuevamente)

In [73]:
ct_falxdep = pd.read_csv('Data/fallecidos_x_departamentos_x_semanasEpi.csv')

In [74]:
def filtering_data_dep(falxdep_df):
    """Function to fix the indexes of the data of deceased by department of Peru.
    IMPORTANT: There are more efficient ways to modify indexes using 'loc' and 
    'iloc' but this method at least 'works'"""

    time = falxdep_df[["fallecido", "Unnamed: 1"]]   # Get the col of epidemiological weeks
    time = time.rename(columns=time.iloc[1])     # Put the first row (epi_week) as header
    time = time.drop([0,1, len(time)-1],axis=0)  # Drop the first and last row (header, nan and total)
    time = time.reset_index(drop=True)           # Reset index

    departments = falxdep_df.drop(["Unnamed: 1", 'fallecido'], axis=1)    # Drop cols that are not departments
    departments = departments.rename(columns=departments.iloc[0])     # Put the first row (epi_week) as header
    departments = departments.drop([0,1, len(departments)-1],axis=0)  # Drop the first and last row (header, nan and total)
    departments = departments.reset_index(drop=True)                  # Reset index

    falxdep_df = pd.concat([time, departments], axis=1)
    return falxdep_df

In [77]:
ct_falxdep_fix = filtering_data_dep(ct_falxdep)
print(ct_falxdep_fix.head())

  epi_year epi_week AMAZONAS ANCASH  ... SAN MARTIN TACNA TUMBES UCAYALI
0     2020       10        0      0  ...          2     0      0       0
1     2020       11        0      0  ...          0     0      0       0
2     2020       12        0      0  ...          0     0      0       2
3     2020       13        0      4  ...          0     0      0       3
4     2020       14        0      0  ...          3     1      3      11

[5 rows x 27 columns]


In [78]:
ct_falxdep_fix.to_csv('Data/fallecidos_x_departamentos_x_semanasEpi.csv', index = False)

# Obteniendo el número **TOTAL de personas completamente vacunadas (2 dosis)** y el número **TOTAL de muertes** confirmadas por COVID-19 por cada uno de los **24 departamentos** del Perú 

## 0. Procesamos el dataset de fallecidos

Realizamos una crosstab del TOTAL de muertes confirmadas por COVID-19 por departamento. Usamos el dataframe "falxdep" usado anteriormente.

In [171]:
dep_deaths = pd.crosstab(index = falxdep['DEPARTAMENTO'], columns = falxdep['fallecido'])
dep_deaths.columns = ['fallecidos']

# Adding an extra row of the total deaths for the whole country
dep_deaths.loc['PERU'] = dep_deaths.sum()

In [108]:
print(dep_deaths)

               fallecidos
DEPARTAMENTO             
AMAZONAS             1273
ANCASH               6728
APURIMAC             1519
AREQUIPA             9718
AYACUCHO             2148
CAJAMARCA            4149
CALLAO              10055
CUSCO                4812
HUANCAVELICA         1176
HUANUCO              2701
ICA                  8569
JUNIN                7061
LA LIBERTAD         10324
LAMBAYEQUE           8642
LIMA                89006
LORETO               4266
MADRE DE DIOS         772
MOQUEGUA             1526
PASCO                1046
PIURA               12163
PUNO                 4228
SAN MARTIN           3030
TACNA                1970
TUMBES               1586
UCAYALI              3059
PERU               201527


## 1. Procesamos el dataset de vacunados

No existe una forma directa de calcular el total de personas completamente vacunadas por departamento. Para lograrlo se prevee lo siguiente:

1. El dataset de vacunación **(RawData/TB_VACUNACION_COVID19.csv)** solo proporciona información sobre el centro de vacunación llamado *id_centro_vacunacion*. NO el departamento u otra ubicación relevante.

2. El dataset de los centros de vacunación **(RawData/TB_CENTRO_VACUNACION.csv)** se puede utilizar para hacer "match" del *id_centro_vacunacion* con el *id_ubigeo*. Que es una variable numérica de 0 a 1894 del departamento.

3. Finalmente, con el dataset de UBIGEO **(RawData/TB_UBIGEOS.csv)** es posible reemplazar a cada *id_ubigeo* con el departamento respectivo.


### 1.1. Obtenemos el total de personas COMPLETAMENTE VACUNADAS (2 dosis) por los 24 departamentos del Perú

In [154]:
def vac_department(vac_url):
    """
    Función que toma la dirección del dataset de vacunados y devuelve el número de VACUNADOS 
    por los 24 departamentos del Perú
    """
    vac_col = ['id_centro_vacunacion', 'dosis']                     
    df_vac = fn.read_largeCSV_file(vac_url, ',', vac_col)    
    lst_vac = fn.df_into_chunks(df_vac)               
    
    # Que cada 'chunk' de lst_vac sea solamente cuando dosis = 2, es decir tengan 2 dosis
    lst_vac = [chunk.loc[chunk.loc[:, 'dosis'] == 2] for chunk in lst_vac]

    # Creamos columnas de 1 en cada chunk para contabilizar cada caso de vacunado
    for chunk in lst_vac:                                        
        chunk['vacunado'] = 1
        chunk['vacunado'].apply(np.int8)
        del chunk['dosis']  # Borramos columna dosis llena de número 2
    del chunk # Borramos el último chunk que queda al final
    
    return lst_vac

In [172]:
vacxdep = vac_department(vac_url)

In [180]:
print(vacxdep[0])

        id_centro_vacunacion  vacunado
0                         62         1
1                        752         1
3                       3138         1
13                         3         1
16                         0         1
...                      ...       ...
499988                     0         1
499989                  1813         1
499991                   705         1
499997                    38         1
499998                    46         1

[220365 rows x 2 columns]


**IMPORTANTE:** Tener en cuenta que cada caso o persona completamente vacunada (fila) tiene una identificación de ubicación. Ahora es necesario recopilar cada caso de vacuna completa con esta identificación. Es decir obtener el TOTAL de vacunados por cada *id_centro_vacunacion*.

### 1.2. De cada caso de vacunado a una crosstab del TOTAL de personas completamente vacunadas (2 dosis) por *id_centro_vacunacion*

**IMPORTANTE:** La variable "vacxdep" es en realidad una lista de *dataframes* o *chunks*.

In [165]:
def vacxdep_chunks(dfs_vac):
    """
    Devuelve un dataframe con el TOTAL DE VACUNADOS por DEPARTAMENTO 
    del PERÚ (recibe una lista de dataframes o chunks)
    """
    var_holder = {}     # Diccionario para guardar nombres                                             
    lst_epi_vac = []    # Lista de dfs para cada sumatoria de chunks
                                         
    for i, chunk in enumerate(dfs_vac):
        var_holder['epi_vac_' + str(i)]= pd.crosstab(index=[chunk['id_centro_vacunacion']],
                                                     columns=chunk['vacunado'])
        lst_epi_vac.append(var_holder['epi_vac_' + str(i)])
    
    merged_epivac = pd.concat(lst_epi_vac, axis=1)  # Merge all dfs
    epi_vac = pd.DataFrame(merged_epivac.sum(numeric_only=True, axis=1))
    epi_vac.columns = ['vacunados']
    epi_vac['vacunados'] = epi_vac['vacunados'].astype(np.int64)
    epi_vac.reset_index(level=0, inplace=True)

    return epi_vac

In [166]:
vacxdep_sum = vacxdep_chunks(vacxdep)

In [170]:
print(vacxdep_sum.head()), print("\n")

print('Totalmente vacunados (2 dosis): ' + str(vacxdep_sum['vacunados'].sum())) 

   id_centro_vacunacion  vacunados
0                     0    5039407
1                     1        344
2                     3     182818
3                     5      10024
4                     6      40121


Totalmente vacunados (2 dosis): 19291391


## 2. Obteniendo el departamento de cada centro de vacunación (*id_centro_vacunacion*)

### 2.1. Cargamos las direccions de RawData

En este caso el dataset de **UBIGEO** y **CENTROS DE VACUNACIÓN**.

In [138]:
ubigeo_url = 'RawData/TB_UBIGEOS.csv'
vaccenter_url = 'RawData/TB_CENTRO_VACUNACION.csv'

### 2.2. Procesamos los datasets

In [139]:
ubigeo = pd.read_csv(ubigeo_url, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(vaccenter_url, usecols= ['id_centro_vacunacion','id_ubigeo'])

vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del vaccenter['id_ubigeo']

print(vaccenter.head(10))

   id_centro_vacunacion departamento
0                  2021         PUNO
1                  3699   SAN MARTIN
2                   154   SAN MARTIN
3                   155   SAN MARTIN
4                  3260   SAN MARTIN
5                  2906       ANCASH
6                  2907       ANCASH
7                  2909       ANCASH
8                  2910       ANCASH
9                  2912       ANCASH


**IMPORTANTE:** Tenga en cuenta que el nuevo dataset **"vaccenter"** se puede utilizar para obtener el departamento de cada centro de vacunación de nuestro anterior dataset llamado **"vacxdep_sum"**.

### 2.2. Encontrar el departamento correspondiente de todas las personas completamente vacunadas (2 dosis)

In [140]:
vacxdep_sum = vacxdep_sum.merge(vaccenter, on = 'id_centro_vacunacion', how = 'left')
del vacxdep_sum["id_centro_vacunacion"]

In [145]:
print(vacxdep_sum.head())

   vacunados departamento
0    5039407     AMAZONAS
1        344         LIMA
2     182818         LIMA
3      10024         LIMA
4      40121         LIMA


### 2.3. Agrupar a todos los vacunados por departamento.

In [146]:
ct_vacxdep = vacxdep_sum.groupby(['departamento']).sum()    # Sum by departments
ct_vacxdep.loc['PERU',:] = ct_vacxdep.sum(axis = 0)         # Total of fully vaccinated
ct_vacxdep['vacunados'] = ct_vacxdep['vacunados'].apply(np.int64) 

In [147]:
print(ct_vacxdep)

               vacunados
departamento            
AMAZONAS         5142271
ANCASH            463675
APURIMAC          149049
AREQUIPA          843156
AYACUCHO          156990
CAJAMARCA         428833
CALLAO            715241
CUSCO               2003
HUANCAVELICA       62718
HUANUCO           163078
ICA               548606
JUNIN             519661
LA LIBERTAD       834518
LAMBAYEQUE        567017
LIMA             6354311
LORETO            259498
MADRE DE DIOS      53349
MOQUEGUA           96511
PASCO              81308
PIURA             694375
PUNO              303010
SAN MARTIN        350882
TACNA             190169
TUMBES             83342
UCAYALI           157200
PERU            19220771


## 3. Unir el dataset del TOTAL de muertes confirmadas y TOTAL vacunados por departamento

In [148]:
vndxdep = pd.concat([ct_vacxdep, dep_deaths], axis=1)
vndxdep.reset_index(level=0, inplace=True)
vndxdep.rename(columns = {'index':'departamento'}, inplace = True)

In [149]:
print(vndxdep)

     departamento  vacunados  fallecidos
0        AMAZONAS    5142271        1273
1          ANCASH     463675        6728
2        APURIMAC     149049        1519
3        AREQUIPA     843156        9718
4        AYACUCHO     156990        2148
5       CAJAMARCA     428833        4149
6          CALLAO     715241       10055
7           CUSCO       2003        4812
8    HUANCAVELICA      62718        1176
9         HUANUCO     163078        2701
10            ICA     548606        8569
11          JUNIN     519661        7061
12    LA LIBERTAD     834518       10324
13     LAMBAYEQUE     567017        8642
14           LIMA    6354311       89006
15         LORETO     259498        4266
16  MADRE DE DIOS      53349         772
17       MOQUEGUA      96511        1526
18          PASCO      81308        1046
19          PIURA     694375       12163
20           PUNO     303010        4228
21     SAN MARTIN     350882        3030
22          TACNA     190169        1970
23         TUMBE

## 4. Procesar el dataset por el número de habitantes correspondiente a cada departamento

Datos de población estimada para el 2020 obtenida del reporte [«PERÚ:Estimaciones y Proyecciones de Población por Departamento, Provincia y Distrito, 2018 - 2020»](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1715/Libro.pdf). Página 39 estimaciones de población total proyectada para 2020. Fuente: [INEI](https://www.inei.gob.pe/).

In [150]:
dic_dep = {
    "AMAZONAS"	:	426806,
    "ANCASH"	:	1180638,
    "APURIMAC"	:	430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:	668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LA LIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADRE DE DIOS":173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SAN MARTIN":	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
    "PERU"  	:	32625948,
}

In [151]:
vndxdep['no_habitantes'] = vndxdep['departamento'].map(dic_dep)
vndxdep = vndxdep[['departamento', 'no_habitantes', 'vacunados', 'fallecidos']]

# Mortality rate per 100k per department
vndxdep['tasa_mortalidad'] = round((vndxdep['fallecidos']/vndxdep['no_habitantes'])*100000,2)

# % of people fully vaccinated per department
vndxdep['vac_porcentaje'] = round((vndxdep['vacunados']*100)/vndxdep['no_habitantes'],2)

In [152]:
print(vndxdep)

     departamento  no_habitantes  ...  tasa_mortalidad  vac_porcentaje
0        AMAZONAS         426806  ...           298.26         1204.83
1          ANCASH        1180638  ...           569.86           39.27
2        APURIMAC         430736  ...           352.65           34.60
3        AREQUIPA        1497438  ...           648.98           56.31
4        AYACUCHO         668213  ...           321.45           23.49
5       CAJAMARCA        1453711  ...           285.41           29.50
6          CALLAO        1129854  ...           889.94           63.30
7           CUSCO        1357075  ...           354.59            0.15
8    HUANCAVELICA         365317  ...           321.91           17.17
9         HUANUCO         760267  ...           355.27           21.45
10            ICA         975182  ...           878.71           56.26
11          JUNIN        1361467  ...           518.63           38.17
12    LA LIBERTAD        2016771  ...           511.91           41.38
13    

Guardamos:

In [21]:
vndxdep.to_csv('Data/TOTAL_vacunados_y_fallecidos_x_departamento.csv',index = False)